# Intelligent Form Agent - Exploration Notebook

This notebook provides an interactive environment to explore the capabilities of the Intelligent Form Agent.

In [ ]:
# Setup - Add project to path
import sys
from pathlib import Path

project_root = Path('.').absolute().parent
sys.path.insert(0, str(project_root))

# Import the agent
from src.agent import IntelligentFormAgent, FormDocument

## Initialize the Agent

In [ ]:
agent = IntelligentFormAgent()
print("Agent initialized successfully!")

## Load Sample Forms

In [ ]:
# Load sample W-2 form
w2_doc = agent.load_form(str(project_root / 'data/sample_forms/sample_w2.txt'))

print(f"Form Type: {w2_doc.file_type}")
print(f"Schema: {w2_doc.schema_type}")
print(f"Confidence: {w2_doc.extraction_confidence:.1%}")
print(f"\nExtracted Fields:")
for field, value in w2_doc.fields.items():
    print(f"  {field}: {value}")

## Question Answering

In [ ]:
# Ask questions about the W-2 form
questions = [
    "What is the employee's name?",
    "How much was the total wages?",
    "What was the federal tax withheld?"
]

for q in questions:
    result = agent.ask(q, w2_doc)
    print(f"Q: {q}")
    print(f"A: {result.answer}")
    print(f"   Confidence: {result.confidence:.1%}")
    print()

## Summarization

In [ ]:
# Generate summary
summary = agent.summarize(w2_doc)
print(summary.full_text)

## Cross-Form Analysis

In [ ]:
# Load multiple onboarding forms
onboarding_docs = agent.load_forms([
    str(project_root / 'data/sample_forms/sample_onboarding_1.txt'),
    str(project_root / 'data/sample_forms/sample_onboarding_2.txt'),
    str(project_root / 'data/sample_forms/sample_onboarding_3.txt')
])

print(f"Loaded {len(onboarding_docs)} forms")

In [ ]:
# Cross-form analysis
analysis = agent.analyze(onboarding_docs, "Compare salaries across departments")

print(f"Documents: {analysis['total_documents']}")
print(f"Common Fields: {analysis['common_fields'][:5]}")
print(f"\nInsights:")
for insight in analysis.get('insights', []):
    print(f"  - {insight}")

## Export Data

In [ ]:
# Export to JSON
import json

json_output = agent.export(w2_doc, 'json')
print("JSON Export:")
print(json.dumps(json.loads(json_output), indent=2)[:500] + "...")

In [ ]:
# Export to Markdown
md_output = agent.export(w2_doc, 'markdown')
print("Markdown Export:")
print(md_output)

## Component Exploration

In [ ]:
# Explore the Schema Detector
from src.processors import SchemaDetector

detector = SchemaDetector()

test_text = """
This is a Form W-2 Wage and Tax Statement.
Employee SSN: 123-45-6789
Federal Income Tax Withheld: $5,000
"""

match = detector.detect_with_confidence(test_text)
print(f"Detected Schema: {match.schema_type}")
print(f"Confidence: {match.confidence:.1%}")
print(f"Matched Indicators: {match.matched_indicators}")

In [ ]:
# Explore the Field Parser
from src.processors import FieldParser

parser = FieldParser()

test_text = """
Name: John Doe
Email: john.doe@example.com
Phone: (555) 123-4567
Amount: $1,234.56
Date: 01/15/2024
[X] Option A
[ ] Option B
"""

fields = parser.parse(test_text)
print("Parsed Fields:")
for field, value in fields.items():
    print(f"  {field}: {value}")

## Custom Configuration

In [ ]:
# Create agent with custom config
import yaml

config = {
    'summarization': {
        'style': 'narrative',
        'max_length': 300
    },
    'qa': {
        'top_k_retrieval': 3
    }
}

# Save config
config_path = project_root / 'notebooks/custom_config.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f)

# Create agent with config
custom_agent = IntelligentFormAgent(config_path=str(config_path))

# Generate narrative summary
narrative_summary = custom_agent.summarize(w2_doc)
print("Narrative Summary:")
print(narrative_summary.full_text)